# 🌍 Africa Gender & Youth Data Tracker
## Notebook 01: Data Ingestion, Cleaning & Validation

**Author:** Teresiah Njoroge  
**Date:** February 2026  
**Project:** AU WGYD M&E Data Pipeline  

---

### Purpose
This notebook ingests raw gender and youth indicator data for 54 African Union 
member states from the World Bank Open Data API. It performs systematic quality 
checks, cleans and validates all datasets, and exports analysis-ready files.

All data quality decisions are documented inline and in `data_cleaning_log.md`.


In [1]:
import wbgapi as wb
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")


✅ Libraries imported successfully


## Step 1: Set Up Folder Structure

We define exact folder paths using absolute paths (full location on disk)
to avoid any folder-not-found errors when saving files.

- `RAW` — where original downloaded files are saved untouched
- `CLEAN` — where cleaned, analysis-ready files are saved

> **Rule:** Always preserve raw data. Never overwrite it.
> If cleaning goes wrong, reload from raw without re-downloading.


In [2]:
# Get exact location of this notebook on your computer
notebook_dir = os.getcwd()

# Build full absolute paths
RAW   = os.path.join(notebook_dir, '01_data', 'raw')
CLEAN = os.path.join(notebook_dir, '01_data', 'cleaned')

# Create folders on disk (exist_ok=True means no error if already exists)
os.makedirs(RAW,   exist_ok=True)
os.makedirs(CLEAN, exist_ok=True)

# Confirm
print("📁 Notebook location:", notebook_dir)
print()
print("RAW folder:  ", RAW)
print("CLEAN folder:", CLEAN)
print()
print("RAW exists:  ", os.path.isdir(RAW))
print("CLEAN exists:", os.path.isdir(CLEAN))


📁 Notebook location: c:\Users\wanji\AppData\Local\Programs\Microsoft VS Code

RAW folder:   c:\Users\wanji\AppData\Local\Programs\Microsoft VS Code\01_data\raw
CLEAN folder: c:\Users\wanji\AppData\Local\Programs\Microsoft VS Code\01_data\cleaned

RAW exists:   True
CLEAN exists: True


## Step 2: Download Raw Data from World Bank API

We pull 5 gender and youth indicators for all 54 AU member states
covering **2015–2023**, aligned with AU Agenda 2063 monitoring targets.

| Indicator Code | Description |
|---|---|
| SL.TLF.ACTI.FE.ZS | Female Labour Force Participation Rate (%) |
| SG.GEN.PARL.ZS | Women in National Parliament (%) |
| SE.SEC.ENRR.FE | Girls' Secondary School Enrolment (%) |
| SL.UEM.1524.FE.ZS | Female Youth Unemployment Rate 15–24 (%) |
| SL.UEM.1524.MA.ZS | Male Youth Unemployment Rate 15–24 (%) |

All datasets are stored in `raw_frames` (a Python dictionary in memory)
AND saved as CSV files to `01_data/raw/` for reproducibility.


In [3]:
indicators = {
    'SL.TLF.ACTI.FE.ZS': 'female_lfp',
    'SG.GEN.PARL.ZS':     'women_in_parliament',
    'SE.SEC.ENRR.FE':     'girls_secondary_enrol',
    'SL.UEM.1524.FE.ZS':  'youth_unemployment_f',
    'SL.UEM.1524.MA.ZS':  'youth_unemployment_m',
}

africa = wb.region.members('AFR')
raw_frames = {}

for code, name in indicators.items():
    print(f"Downloading {name}...")
    df = wb.data.DataFrame(code,
                            economy=africa,
                            time=range(2015, 2024))
    df.reset_index(inplace=True)
    raw_frames[name] = df
    print(f"  ✅ Shape: {df.shape}")

print()
print("✅ raw_frames contains:", list(raw_frames.keys()))


  ✅ Shape: (54, 10)
  ✅ Shape: (54, 10)
  ✅ Shape: (54, 10)
  ✅ Shape: (54, 10)
  ✅ Shape: (54, 10)

✅ raw_frames contains: ['female_lfp', 'women_in_parliament', 'girls_secondary_enrol', 'youth_unemployment_f', 'youth_unemployment_m']


## Step 3: Save Raw Files to Disk

We save each downloaded dataset as a CSV file **before any cleaning begins**.

> **Why save raw files?**  
> This is standard data practice — always preserve the original untouched data.  
> If cleaning goes wrong at any point, you can reload from raw  
> without having to re-download from the internet.


In [4]:
for name, df in raw_frames.items():
    save_path = os.path.join(RAW, f'{name}.csv')
    df.to_csv(save_path, index=False)
    print(f"✅ Saved: {name}.csv")

print()
print("📂 Files now in RAW folder:")
for f in os.listdir(RAW):
    print(f"   {f}")


✅ Saved: female_lfp.csv
✅ Saved: women_in_parliament.csv
✅ Saved: girls_secondary_enrol.csv
✅ Saved: youth_unemployment_f.csv
✅ Saved: youth_unemployment_m.csv

📂 Files now in RAW folder:
   female_lfp.csv
   girls_secondary_enrol.csv
   women_in_parliament.csv
   youth_unemployment_f.csv
   youth_unemployment_m.csv


## Step 4: Data Quality Assessment

Before cleaning anything, we run a quality check on every raw dataset.
We check for 4 things:

| Check | Question we are asking |
|---|---|
| **Shape** | How many rows and columns does this dataset have? |
| **Missing values** | How many empty cells are there in total? |
| **Duplicates** | Are any rows repeated exactly? |
| **Countries** | How many unique countries are included? |

This step tells us **what problems exist** before we fix them.  
We expect 54 countries per indicator (all AU member states).


In [5]:
def quality_report(df, name):
    print(f"\n{'='*50}")
    print(f"DATASET: {name}")
    print(f"  Shape:          {df.shape}")
    print(f"  Missing values: {df.isnull().sum().sum()} total cells")
    print(f"  Duplicate rows: {df.duplicated().sum()}")
    if 'economy' in df.columns:
        print(f"  Countries:      {df['economy'].nunique()}")

for name, df in raw_frames.items():
    quality_report(df, name)

print(f"\n{'='*50}")
print("✅ Quality check complete")



DATASET: female_lfp
  Shape:          (54, 10)
  Missing values: 10 total cells
  Duplicate rows: 0
  Countries:      54

DATASET: women_in_parliament
  Shape:          (54, 10)
  Missing values: 16 total cells
  Duplicate rows: 0
  Countries:      54

DATASET: girls_secondary_enrol
  Shape:          (54, 10)
  Missing values: 270 total cells
  Duplicate rows: 0
  Countries:      54

DATASET: youth_unemployment_f
  Shape:          (54, 10)
  Missing values: 10 total cells
  Duplicate rows: 0
  Countries:      54

DATASET: youth_unemployment_m
  Shape:          (54, 10)
  Missing values: 10 total cells
  Duplicate rows: 0
  Countries:      54

✅ Quality check complete


## Step 5: Data Cleaning & Standardisation

We apply 8 cleaning steps to every dataset:

| Step | Action | Reason |
|---|---|---|
| 1 | Identify year vs non-year columns | Separate data from metadata |
| 2 | Reshape wide → long format | One row per country per year (tidy data) |
| 3 | Clean year column | Extract 4-digit year from formats like `YR2015` |
| 4 | Rename `economy` → `country_code` | Standardise column naming |
| 5 | Add AU region | Enable regional comparisons |
| 6 | Add `quality_flag` | Mark MISSING vs VERIFIED — never silently drop data |
| 7 | Add indicator name | Identify which indicator each row belongs to |
| 8 | Remove duplicates | One record per country per year only |

> **Key decision:** Missing values are FLAGGED, not deleted or imputed.  
> This is standard M&E practice — gaps must be visible in reports,  
> not hidden by automatic substitution.

> **Notable finding from quality check:**  
> `girls_secondary_enrol` has 270 missing cells — this will be  
> flagged in the data cleaning log and reported to stakeholders.


In [6]:
au_regions = {
    'KEN':'East Africa','ETH':'East Africa','TZA':'East Africa',
    'UGA':'East Africa','RWA':'East Africa','BDI':'East Africa',
    'SOM':'East Africa','DJI':'East Africa','ERI':'East Africa',
    'SSD':'East Africa','MDG':'East Africa','COM':'East Africa',
    'MUS':'East Africa','SYC':'East Africa',
    'NGA':'West Africa','GHA':'West Africa','SEN':'West Africa',
    'CIV':'West Africa','MLI':'West Africa','BFA':'West Africa',
    'NER':'West Africa','GIN':'West Africa','SLE':'West Africa',
    'LBR':'West Africa','MRT':'West Africa','GMB':'West Africa',
    'GNB':'West Africa','CPV':'West Africa','TGO':'West Africa',
    'BEN':'West Africa',
    'ZAF':'Southern Africa','ZWE':'Southern Africa','MOZ':'Southern Africa',
    'ZMB':'Southern Africa','MWI':'Southern Africa','BWA':'Southern Africa',
    'NAM':'Southern Africa','LSO':'Southern Africa','SWZ':'Southern Africa',
    'AGO':'Central Africa','CMR':'Central Africa','COD':'Central Africa',
    'COG':'Central Africa','CAF':'Central Africa','TCD':'Central Africa',
    'GNQ':'Central Africa','GAB':'Central Africa','STP':'Central Africa',
    'EGY':'North Africa','DZA':'North Africa','MAR':'North Africa',
    'TUN':'North Africa','LBA':'North Africa','SDN':'North Africa',
}

def clean_dataset(df, indicator_name):
    # Step 1: Identify year columns vs non-year columns
    year_cols = [c for c in df.columns
                 if str(c).startswith('YR') or
                 (str(c).replace('.','').isdigit() and len(str(c)) == 4)]
    id_cols = [c for c in df.columns if c not in year_cols]

    # Step 2: Reshape wide to long — one row per country per year
    df_long = df.melt(id_vars=id_cols,
                      value_vars=year_cols,
                      var_name='year',
                      value_name='value')

    # Step 3: Clean year column — extract 4-digit number
    df_long['year'] = df_long['year'].astype(str).str.extract(r'(\d{4})').astype(int)

    # Step 4: Rename economy to country_code
    if 'economy' in df_long.columns:
        df_long.rename(columns={'economy': 'country_code'}, inplace=True)

    # Step 5: Add AU region
    df_long['au_region'] = df_long['country_code'].map(au_regions).fillna('Other')

    # Step 6: Add quality flag
    df_long['quality_flag'] = np.where(
        df_long['value'].isnull(), 'MISSING', 'VERIFIED'
    )

    # Step 7: Add indicator name
    df_long['indicator'] = indicator_name

    # Step 8: Remove duplicates
    df_long.drop_duplicates(subset=['country_code', 'year'], inplace=True)
    df_long.reset_index(drop=True, inplace=True)

    return df_long

print("✅ clean_dataset function defined — ready to run")


✅ clean_dataset function defined — ready to run


## Step 6: Run Cleaning & Save Cleaned Files

We now apply the `clean_dataset` function to all 5 indicators
and save the cleaned outputs to `01_data/cleaned/`.

After this step each cleaned file will have these columns:

| Column | Description |
|---|---|
| `country_code` | 3-letter ISO country code (e.g. KEN, ETH) |
| `year` | 4-digit year (2015–2023) |
| `value` | Indicator value for that country and year |
| `au_region` | AU regional classification |
| `quality_flag` | VERIFIED = has data, MISSING = empty cell |
| `indicator` | Name of the indicator |


In [7]:
cleaned_frames = {}

for name, df in raw_frames.items():
    print(f"Cleaning {name}...")

    # Run the cleaning function
    cleaned = clean_dataset(df.copy(), name)

    # Store in memory
    cleaned_frames[name] = cleaned

    # Save to cleaned folder using absolute path
    save_path = os.path.join(CLEAN, f'{name}_clean.csv')
    cleaned.to_csv(save_path, index=False)

    # Summary counts
    verified = (cleaned['quality_flag'] == 'VERIFIED').sum()
    missing  = (cleaned['quality_flag'] == 'MISSING').sum()
    total    = len(cleaned)
    pct      = round(verified / total * 100, 1)

    print(f"  ✅ Rows: {total} | Verified: {verified} | Missing: {missing} | Completeness: {pct}%")

print()
print("📂 Files saved to CLEAN folder:")
for f in os.listdir(CLEAN):
    print(f"   {f}")

print()
print("✅ All datasets cleaned and saved successfully")


Cleaning female_lfp...
  ✅ Rows: 486 | Verified: 476 | Missing: 10 | Completeness: 97.9%
Cleaning women_in_parliament...
  ✅ Rows: 486 | Verified: 470 | Missing: 16 | Completeness: 96.7%
Cleaning girls_secondary_enrol...
  ✅ Rows: 486 | Verified: 216 | Missing: 270 | Completeness: 44.4%
Cleaning youth_unemployment_f...
  ✅ Rows: 486 | Verified: 476 | Missing: 10 | Completeness: 97.9%
Cleaning youth_unemployment_m...
  ✅ Rows: 486 | Verified: 476 | Missing: 10 | Completeness: 97.9%

📂 Files saved to CLEAN folder:
   female_lfp_clean.csv
   girls_secondary_enrol_clean.csv
   women_in_parliament_clean.csv
   youth_unemployment_f_clean.csv
   youth_unemployment_m_clean.csv

✅ All datasets cleaned and saved successfully
